# Creating and Appending to an Icechunk Store with Virtual References
This notebook demonstrates how to create an icechunk store and then append to it.

See this blog post for more info:  https://tom-nicholas.com/blog/2025/cloud-optimized-scientific-data

In [1]:
import warnings
import os
import fsspec
import icechunk
import xarray as xr
from obstore.store import from_url

from virtualizarr import open_virtual_dataset
from virtualizarr.parsers import HDFParser
from virtualizarr.registry import ObjectStoreRegistry

warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
import virtualizarr
print(icechunk.__version__)
print(virtualizarr.__version__)

1.1.9
2.1.1


In [3]:
# load AWS credentials for Pangeo-EOSC storage as environment vars
from dotenv import load_dotenv
_ = load_dotenv(f'{os.environ['HOME']}/dotenv/school_2025.env')

username = os.environ['JUPYTERHUB_USER']

# Define storage
storage_endpoint = 'https://pangeo-eosc-minioapi.vm.fedcloud.eu'
data_bucket = 'rsignell4-protocoast'
storage_bucket = 'protocoast-school-2025'
storage_name = f'{username}-taranto'

### Create a list of files that will make up the virtual dataset
First we will handle the "nos" SHYFEM files

In [4]:
fs = fsspec.filesystem('s3', anon=True, endpoint_url=storage_endpoint)
flist = fs.glob(f'{data_bucket}/full_dataset/shyfem/taranto/forecast/*/*nos*.nc')
flist = [f's3://{f}' for f in flist]

In [5]:
print(len(flist))
print(flist[-1])

19
s3://rsignell4-protocoast/full_dataset/shyfem/taranto/forecast/20251001/taranto_nos_20251001_nc4.nc


### Try opening one of these NetCDF files

In [6]:
ds = xr.open_dataset(fs.open(flist[0]))

In [7]:
ds['salinity'][:5,100,0].values

array([38.831997, 38.834663, 38.83676 , 38.834667, 38.83502 ],
      dtype=float32)

In [8]:
ds

<xarray.Dataset> Size: 2GB
Dimensions:        (element: 58285, vertex: 3, node: 30731, time: 144, level: 70)
Coordinates:
  * time           (time) datetime64[ns] 1kB 2025-09-08 ... 2025-09-13T23:00:00
  * level          (level) float32 280B 2.0 4.0 6.0 ... 1.5e+03 2e+03 2.5e+03
Dimensions without coordinates: element, vertex, node
Data variables:
    element_index  (element, vertex) int32 699kB ...
    latitude       (node) float32 123kB ...
    longitude      (node) float32 123kB ...
    salinity       (time, node, level) float32 1GB ...
    temperature    (time, node, level) float32 1GB ...
    topology       int32 4B ...
    total_depth    (node) float32 123kB ...
Attributes:
    Conventions:  CF-1.4
    title:        uae
    history:      Wed Sep 10 15:12:42 2025: ncks -4 -L 5 -O -d time,0,143 --c...
    institution:  Centro Euro-Mediterraneo sui Cambiamenti Climatici - CMCC, ...
    source:       Model data produced by SHYFEM-MPI at CMCC
    references:   https://zenodo.org/record/5596734#.Y-YwpxPMLx8
    NCO:          netCDF Operators version 5.0.6 (Homepage = http://nco.sf.ne...

### Define our Virtualizarr `Parser` and `ObjectStoreRegistry`

In [9]:
bucket = f"s3://{data_bucket}"
store = from_url(bucket, region="not-used", endpoint=storage_endpoint, skip_signature=True)
registry = ObjectStoreRegistry({bucket: store})
parser = HDFParser()

## Create virtual datasets from each file with VirtualiZarr's `open_virtual_dataset`

In [15]:
%%time
ds_list = [
    open_virtual_dataset(
        url=url,
        parser=parser,
        registry=registry,
        loadable_variables=["time"],
    )
    for url in flist[0:-1]
]

CPU times: user 7.14 s, sys: 8.33 s, total: 15.5 s
Wall time: 27.8 s


### "fix" each dataset to match the requirements of the "Rolodex" FMRC indexing

In [16]:
def fix_ds(ds):
    ds = ds.rename_vars(time='valid_time')
    ds = ds.rename_dims(time='step')
    step = (ds.valid_time - ds.valid_time[0]).assign_attrs({"standard_name": "forecast_period"})
    time = ds.valid_time[0].assign_attrs({"standard_name": "forecast_reference_time"})
    ds = ds.assign_coords(step=step, time=time)
    ds = ds.drop_indexes("valid_time")
    ds = ds.drop_vars('valid_time')
    ds = ds.set_coords(['latitude', 'longitude', 'element_index', 'topology', 'total_depth'])
    return ds

In [17]:
ds_list = [fix_ds(ds) for ds in ds_list]

In [18]:
combined_nos = xr.concat(
    ds_list,
    dim="time",
    coords="minimal",
    compat="override",
    combine_attrs="override",
)

In [19]:
combined_nos

<xarray.Dataset> Size: 45GB
Dimensions:        (time: 18, step: 144, node: 30731, level: 70,
                    element: 58285, vertex: 3)
Coordinates:
  * time           (time) datetime64[ns] 144B 2025-09-08 ... 2025-09-29
  * step           (step) timedelta64[ns] 1kB 00:00:00 ... 5 days 23:00:00
    level          (level) float32 280B ManifestArray<shape=(70,), dtype=floa...
    element_index  (element, vertex) int32 699kB ManifestArray<shape=(58285, ...
    latitude       (node) float32 123kB ManifestArray<shape=(30731,), dtype=f...
    longitude      (node) float32 123kB ManifestArray<shape=(30731,), dtype=f...
    topology       int32 4B ManifestArray<shape=(), dtype=int32, chunks=()>
    total_depth    (node) float32 123kB ManifestArray<shape=(30731,), dtype=f...
Dimensions without coordinates: node, element, vertex
Data variables:
    salinity       (time, step, node, level) float32 22GB ManifestArray<shape...
    temperature    (time, step, node, level) float32 22GB ManifestArray<shape...
Attributes:
    Conventions:  CF-1.4
    title:        uae
    history:      Wed Sep 10 15:12:42 2025: ncks -4 -L 5 -O -d time,0,143 --c...
    institution:  Centro Euro-Mediterraneo sui Cambiamenti Climatici - CMCC, ...
    source:       Model data produced by SHYFEM-MPI at CMCC
    references:   https://zenodo.org/record/5596734#.Y-YwpxPMLx8
    NCO:          netCDF Operators version 5.0.6 (Homepage = http://nco.sf.ne...

Now we handle the "ous" SHYFEM files

In [20]:
flist = fs.glob(f'{data_bucket}/full_dataset/shyfem/taranto/forecast/*/*ous*.nc')
flist = [f's3://{f}' for f in flist]

In [21]:
%%time
ds_list = [
    open_virtual_dataset(
        url=url,
        parser=parser,
        registry=registry,
        loadable_variables=["time"],
    )
    for url in flist[0:-1]
]

CPU times: user 7.66 s, sys: 9.32 s, total: 17 s
Wall time: 32.4 s


In [22]:
ds_list = [fix_ds(ds) for ds in ds_list]

In [23]:
combined_ous = xr.concat(
    ds_list,
    dim="time",
    coords="minimal",
    compat="override",
    combine_attrs="override",
)

### Now create a virtual dataset with the "nos" and "ous" datasets merged together

In [24]:
ds = xr.merge([combined_nos, combined_ous], compat='override')

In [25]:
ds

<xarray.Dataset> Size: 90GB
Dimensions:        (time: 18, step: 144, node: 30731, level: 70,
                    element: 58285, vertex: 3)
Coordinates:
  * time           (time) datetime64[ns] 144B 2025-09-08 ... 2025-09-29
  * step           (step) timedelta64[ns] 1kB 00:00:00 ... 5 days 23:00:00
    level          (level) float32 280B ManifestArray<shape=(70,), dtype=floa...
    element_index  (element, vertex) int32 699kB ManifestArray<shape=(58285, ...
    latitude       (node) float32 123kB ManifestArray<shape=(30731,), dtype=f...
    longitude      (node) float32 123kB ManifestArray<shape=(30731,), dtype=f...
    topology       int32 4B ManifestArray<shape=(), dtype=int32, chunks=()>
    total_depth    (node) float32 123kB ManifestArray<shape=(30731,), dtype=f...
Dimensions without coordinates: node, element, vertex
Data variables:
    salinity       (time, step, node, level) float32 22GB ManifestArray<shape...
    temperature    (time, step, node, level) float32 22GB ManifestArray<shape...
    u_velocity     (time, step, node, level) float32 22GB ManifestArray<shape...
    v_velocity     (time, step, node, level) float32 22GB ManifestArray<shape...
    water_level    (time, step, node) float32 319MB ManifestArray<shape=(18, ...
Attributes:
    Conventions:  CF-1.4
    title:        uae
    history:      Wed Sep 10 15:12:42 2025: ncks -4 -L 5 -O -d time,0,143 --c...
    institution:  Centro Euro-Mediterraneo sui Cambiamenti Climatici - CMCC, ...
    source:       Model data produced by SHYFEM-MPI at CMCC
    references:   https://zenodo.org/record/5596734#.Y-YwpxPMLx8
    NCO:          netCDF Operators version 5.0.6 (Homepage = http://nco.sf.ne...

## Initialize the Icechunk Store
We need configure the `virtual_chunk_container` as make sure the icechunk container credentials allow for anonymous access. 
Details on this can be found [here](https://icechunk.io/en/stable/virtual/).

In [26]:
fs_school = fsspec.filesystem('s3', anon=True, endpoint_url=storage_endpoint)

In [27]:
# remove old existing icechunk storage with this name
try:
    # Use the same prefix as the storage
    fs_school.rm(f's3://{storage_bucket}/icechunk/{storage_name}', recursive=True)
except:
    pass

In [28]:
fs_school.ls(f'{storage_bucket}')

['protocoast-school-2025/rsignell4']

In [29]:
storage = icechunk.s3_storage(
    bucket=storage_bucket,
    prefix=f"icechunk/{storage_name}",
    anonymous=True,
    endpoint_url=storage_endpoint,
    region='not-used',   # N/A for Pangeo-EOSC bucket, but required param
    force_path_style=True)

In [30]:
config = icechunk.RepositoryConfig.default()

config.set_virtual_chunk_container(
    icechunk.VirtualChunkContainer(
        url_prefix=f"s3://{data_bucket}/",
        store=icechunk.s3_store(region="not-used", anonymous=True, s3_compatible=True, 
                                force_path_style=True, endpoint_url=storage_endpoint),
    ),
)

repo = icechunk.Repository.create(storage, config)
session = repo.writable_session("main")

## Write the virtual datasets to the icechunk store and commit

In [31]:
ds.virtualize.to_icechunk(session.store)

In [32]:
session.commit("all but last day of Taranto data")

'X5F5SCEW7XCQYDFNXPSG'

## Check the icechunk dataset

In [33]:
credentials = icechunk.containers_credentials(
    {f"s3://{data_bucket}/": icechunk.s3_credentials(anonymous=True)})

read_repo = icechunk.Repository.open(
    storage, config, authorize_virtual_chunk_access=credentials)

read_session = read_repo.readonly_session("main")

In [34]:
ds = xr.open_zarr(read_session.store, consolidated=False, zarr_format=3)
ds

<xarray.Dataset> Size: 90GB
Dimensions:        (time: 18, step: 144, node: 30731, level: 70,
                    element: 58285, vertex: 3)
Coordinates:
  * time           (time) datetime64[ns] 144B 2025-09-08 ... 2025-09-29
  * step           (step) timedelta64[ns] 1kB 00:00:00 ... 5 days 23:00:00
  * level          (level) float32 280B 2.0 4.0 6.0 ... 1.5e+03 2e+03 2.5e+03
    element_index  (element, vertex) int32 699kB dask.array<chunksize=(58285, 3), meta=np.ndarray>
    latitude       (node) float32 123kB dask.array<chunksize=(16000,), meta=np.ndarray>
    topology       int32 4B ...
    longitude      (node) float32 123kB dask.array<chunksize=(16000,), meta=np.ndarray>
    total_depth    (node) float32 123kB dask.array<chunksize=(16000,), meta=np.ndarray>
Dimensions without coordinates: node, element, vertex
Data variables:
    temperature    (time, step, node, level) float32 22GB dask.array<chunksize=(1, 72, 16000, 1), meta=np.ndarray>
    salinity       (time, step, node, level) float32 22GB dask.array<chunksize=(1, 72, 16000, 1), meta=np.ndarray>
    v_velocity     (time, step, node, level) float32 22GB dask.array<chunksize=(1, 72, 16000, 1), meta=np.ndarray>
    water_level    (time, step, node) float32 319MB dask.array<chunksize=(1, 72, 16000), meta=np.ndarray>
    u_velocity     (time, step, node, level) float32 22GB dask.array<chunksize=(1, 72, 16000, 1), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.4
    title:        uae
    history:      Wed Sep 10 15:12:42 2025: ncks -4 -L 5 -O -d time,0,143 --c...
    institution:  Centro Euro-Mediterraneo sui Cambiamenti Climatici - CMCC, ...
    source:       Model data produced by SHYFEM-MPI at CMCC
    references:   https://zenodo.org/record/5596734#.Y-YwpxPMLx8
    NCO:          netCDF Operators version 5.0.6 (Homepage = http://nco.sf.ne...

In [35]:
ds = xr.open_dataset(
    read_session.store,
    engine='zarr',
    backend_kwargs={'consolidated': False, 'zarr_format': 3}
)

In [36]:
ds

<xarray.Dataset> Size: 90GB
Dimensions:        (time: 18, step: 144, node: 30731, level: 70,
                    element: 58285, vertex: 3)
Coordinates:
  * time           (time) datetime64[ns] 144B 2025-09-08 ... 2025-09-29
  * step           (step) timedelta64[ns] 1kB 00:00:00 ... 5 days 23:00:00
  * level          (level) float32 280B 2.0 4.0 6.0 ... 1.5e+03 2e+03 2.5e+03
    latitude       (node) float32 123kB ...
    element_index  (element, vertex) int32 699kB ...
    total_depth    (node) float32 123kB ...
    longitude      (node) float32 123kB ...
    topology       int32 4B ...
Dimensions without coordinates: node, element, vertex
Data variables:
    u_velocity     (time, step, node, level) float32 22GB ...
    v_velocity     (time, step, node, level) float32 22GB ...
    water_level    (time, step, node) float32 319MB ...
    temperature    (time, step, node, level) float32 22GB ...
    salinity       (time, step, node, level) float32 22GB ...
Attributes:
    Conventions:  CF-1.4
    title:        uae
    history:      Wed Sep 10 15:12:42 2025: ncks -4 -L 5 -O -d time,0,143 --c...
    institution:  Centro Euro-Mediterraneo sui Cambiamenti Climatici - CMCC, ...
    source:       Model data produced by SHYFEM-MPI at CMCC
    references:   https://zenodo.org/record/5596734#.Y-YwpxPMLx8
    NCO:          netCDF Operators version 5.0.6 (Homepage = http://nco.sf.ne...